# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [31]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [64]:
# your code here

quatil = orders.amount_spent.quantile(0.95)
a = orders[orders['amount_spent'] >= quatil].groupby('CustomerID').agg({'amount_spent':sum})
a
labels = ['VIP', 'Preferred']
a = pd.qcut(a['amount_spent'],2, labels = labels)
a = pd.DataFrame(a)
a


,amount_spent
CustomerID,
12346,Preferred
12347,Preferred
12348,Preferred
12349,Preferred
12352,Preferred
...,...
18260,VIP
18272,VIP
18273,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [103]:
# your code here
VIPS = a[a['amount_spent'] == 'VIP']
VIPS

#indexes = VIPS.index
#indexes = list(indexes)
#gc = orders.groupby('CustomerID').agg({'amount_spent':sum})
#gc['Country'] = orders['Country']
#gc['Count'] = 1
#gc[gc.index.isin(indexes)]
#country_count = gc.groupby('Country').agg({'Count':sum})
#country_count = country_count.sort_values(by='Count', ascending=False)
#country_count

vip = VIPS.groupby('Country').agg({'Count':sum})
vip

,Count
Country,
Channel Islands,1
Denmark,3
EIRE,12
France,15
Germany,16
Japan,4
Norway,16
Portugal,5
Spain,6


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [96]:
# your code here
a['Count'] = 1
a['Country'] = orders['Country']

vip_pf = a.groupby('Country').agg({'Count':sum})
vip_pf

,Count
Country,
Channel Islands,3
Denmark,7
EIRE,31
France,27
Germany,36
Japan,10
Norway,36
Portugal,11
Spain,19
